# Swim Transformer : Hierarchical Vision Transformer using Shifted Windows

在2021年微软亚洲研究院基于Transformer架构提出了一种通用的视觉backnone，使用分层窗口注意力机制移位窗口注意力计算等方法解决了原有的Transformer因为模型庞大难以应用在目标检测和语义分割这种像素级任务上的缺陷。

目前Transformer应用到图像领域主要有两大挑战：

- 视觉实体变化大，在不同场景下视觉Transformer性能未必很好
- 图像分辨率高，像素点多，Transformer基于全局自注意力的计算导致计算量较大

针对上述两个问题，我们提出了一种**包含滑窗操作，具有层级设计**的Swin Transformer。

其中滑窗操作包括**不重叠的local window，和重叠的cross-window**。将注意力计算限制在一个窗口中，**一方面能引入CNN卷积操作的局部性，另一方面能节省计算量。**

## 模型架构


<div align = "center">
    <img src = ./images/swim-tfm-0.png width = 90%>
</div>

整个模型采取层次化的设计，一共包含4个Stage，每个stage都会缩小输入特征图的分辨率，翻倍通道数，像CNN一样逐层扩大感受野。

与Vit相比，Swim-tfm有以下几点区别：

- 在模型开头取消Position embedding和Class token的操作，直接进行patch分割
- 从Stage2开始，最开始都进行Patch Merging，用于降低图片分辨率
- MSA修改为Window MSA以及Shifted window MSA，且先经过W-MSA再经过SW-MSA

参考[微软官方代码](https://github.com/microsoft/Swin-Transformer)，进行理解。

```python
class SwinTransformer(nn.Module):
    def __init__(...):
        super().__init__()
        ...
        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            
        self.pos_drop = nn.Dropout(p=drop_rate)

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(...)
            self.layers.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        x = self.patch_embed(x)
        if self.ape:
            x = x + self.absolute_pos_embed
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)  # B L C
        x = self.avgpool(x.transpose(1, 2))  # B C 1
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x
```
